In [1]:
import pandas as pd 
import torch
import os

In [2]:
df = pd.read_csv("../data/test_data.csv", header= 0)
df["file_name"] = df["file_name"].map(lambda x : x.split(".")[0] + ".npy")
df = df[df["file_name"].isin(os.listdir("../data/embeddings"))]

In [3]:
df["pred"] = torch.load("modles/predictions_ways-45_lr-0.0001_wd-0.0005.pt", map_location=torch.device('cpu')).tolist()

In [6]:
df["consistency"] = df["consistency"].map({"Consistent": 0, "Inconsistent": 1})

In [15]:
df["pred"] = df["pred"].map({0.0 : 1, 1.0:0})

In [16]:
df[["consistency", "pred"]]

,consistency,pred
0,0,1
1,0,1
2,0,0
3,0,0
4,0,1
...,...,...
1253,1,1
1254,1,1
1255,1,1
1256,1,1


In [19]:
PREFIXES = ["AIO","EID","OIL","OILWI","OILWPI","UE","UEWI1","UEWI2","UEWPI","UEWIP","SOSINETO","CSC","OOR","OOD"]

def get_misclassified_percentage(pattern : str):
    correct = 0
    total = 0
    df_pattern = df[df["injected_pattern"] == pattern]
    print(f"Pattern {pattern}: {len(df_pattern[df_pattern['pred'] == 1])} /{len(df_pattern)}")

for pattern in PREFIXES:
    get_misclassified_percentage(pattern)

Pattern AIO: 64 /81
Pattern EID: 165 /221
Pattern OIL: 1 /1
Pattern OILWI: 0 /1
Pattern OILWPI: 0 /0
Pattern UE: 71 /82
Pattern UEWI1: 9 /10
Pattern UEWI2: 9 /10
Pattern UEWPI: 1 /1
Pattern UEWIP: 3 /3
Pattern SOSINETO: 1 /1
Pattern CSC: 171 /210
Pattern OOR: 5 /5
Pattern OOD: 4 /4


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [17]:
accuracy = accuracy_score(df["consistency"], df["pred"])
precision = precision_score(df["consistency"], df["pred"], zero_division=1)
recall = recall_score(df["consistency"], df["pred"], zero_division=1)

In [18]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.4876
Precision: 0.4941
Recall: 0.8000


In [21]:
def false_positive_rate(y_true, y_pred):
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    total_negatives = (y_true == 0).sum()
    return (fp / total_negatives) * 100 if total_negatives > 0 else 0
fp_rate = false_positive_rate(df["consistency"], df["pred"])

print(f"False Positive Rate: {fp_rate:.2f}%")


False Positive Rate: 82.83%
